In [ ]:
import numpy as np
import h5py
from pathlib import Path
from extra_geom import AGIPD_1MGeometry
from cxiapi import cxiData, calibrateModule
from cxiapi.hitAnalyzer import hitAnalyzer
import matplotlib.pyplot as plt
from p_tqdm import p_umap, p_map
from functools import partial
import multiprocessing as mp

In [ ]:
# Experiment run number.
run = 364
# Hitfinding on which module.
module = 15
# The folder of cxi files
cxi_folder = '/gpfs/exfel/u/scratch/SPB/202130/p900201/spi-comission/vds/'
# Cheetah files folder for calibration
calib_folder = '/gpfs/exfel/exp/SPB/202130/p900201/usr/Software/calib/r0361-r0362-r0363/'
# Geometry file for the detector
geom_file = '/gpfs/exfel/exp/SPB/202130/p900201/usr/Software/geom/agipd_2696_v5.geom'

In [ ]:
cxi_path = Path(cxi_folder, f'r{run:04}.cxi')
fn = str(cxi_path)

cxi = cxiData(fn, verbose=1, debug=0)
pulse = np.arange(0, 352)
base_pulse_filter = np.ones(600, dtype="bool")
base_pulse_filter[len(pulse):] = False
base_pulse_filter[0] = False
base_pulse_filter[18::32] = False
base_pulse_filter[29::32] = False
good_cells = pulse[base_pulse_filter[:len(pulse)]]
cxi.setGoodCells(good_cells)
cxi.setCalib(calib_folder)
cxi.setGainMode(0)

In [ ]:
hits_analyzer = hitAnalyzer(cxi_data = cxi, verbose=1)
hits_analyzer.setAdu_per_photon(45)
# ROI
ROI = (slice(512-50,None), slice(None,51))
hits_analyzer.setROI(ROI)
# Mask
mask = np.ones((512,128))
hits_analyzer.setModuleMasks(15,mask)
hits_analyzer.plotROI(300,15,is_transpose=True)
plt.title('Before mask')

mask[470:473,15:18] = 0
hits_analyzer.setModuleMasks(15,mask)
hits_analyzer.plotROI(300,15,is_transpose=True)
plt.title('After mask')

In [ ]:
data = cxi.data
good_frames = cxi.good_frames
cell_ids = cxi.cellIDs
module_index = 15
intens_thresh = 5
module_mask = hits_analyzer.module_masks[str(module_index)]
ROI = hits_analyzer.ROI
adu_per_photon = hits_analyzer.adu_per_photon

In [ ]:
def getHits(idx_range: list) -> list:
    # Get the index of the hits
    num_cpus = checkChuck(len(idx_range), chuckSize=25000)
    print(f'Using {num_cpus} CPU cores.')
    results = p_umap(partial(check_snapshot,module_index=15), idx_range, num_cpus=num_cpus)
    hits_indices = list(filter((None).__ne__, results))
    return  hits_indices

def getCalibData(cxi_data, snap_idx, module_index):
    n = snap_idx
    data = cxi_data.data
    cell_ids = cxi_data.cellIDs

    calib_data = calibrateModule(data[n, module_index, 0, :, :],
                                    data[n, module_index,
                                        1, :, :], module_index,
                                    cell_ids[n, 0], cxi_data.calib)
    return calib_data

def check_snapshot(snap_idx, module_index):
    calib_data = getCalibData(cxi, snap_idx, module_index)
    roi_data = calib_data[ROI]
    nphotons = roi_data / adu_per_photon
    nphotons[nphotons < 0.5] = 0
    if intens_thresh < np.log(nphotons.sum()):
        return snap_idx
    
def checkChuck(ntask, chuckSize):
    num_cpu = min(mp.cpu_count(), ntask // chuckSize)
    num_cpu = max(num_cpu, 1)
    return num_cpu

In [ ]:
idx_range = cxi.good_frames
module_index = 15
intens_thresh = 5
module_mask = hits_analyzer.module_masks[str(module_index)]
ROI = hits_analyzer.ROI
adu_per_photon = hits_analyzer.adu_per_photon

# hits_indices = getHits(idx_range, module_index,
#             intens_thresh, module_mask, ROI, adu_per_photon)
hits_indices = getHits(idx_range)